In [1]:
pip install --upgrade google-cloud-vision

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Imports the Google Cloud client library
from google.cloud import vision
from io import BytesIO
from tqdm import tqdm
import pandas as pd

In [3]:
def run_quickstart(image_bytes: bytes) -> str:
    """Performs OCR on an image (provided as bytes) using the Cloud Vision API.

    Args:
        image_bytes: The byte content of the image.

    Returns:
        The text detected in the image, or an empty string on error.
    """
    client = vision.ImageAnnotatorClient()

    image = vision.Image(content=image_bytes)

    try:
        response = client.text_detection(image=image)
        text_annotations = response.text_annotations
        if text_annotations:
            return text_annotations[0].description.strip()  # Return the full text, stripped
        else:
            return ""  # Return empty string if no text is found
    except Exception as e:
        print(f"Error during OCR: {e}")
        return ""  # Return empty string on error

In [ ]:
# Replace with your actual values
project_id = 'project_id'
location = "us"
processor_id = "processor_id"
file_path = "example_text_ransaka.jpg"

In [5]:
from datasets import load_dataset

/Users/nevidujayatilleke/Documents/MSC - Research/OCR Comparative Analysis/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
dataset = load_dataset("Nevidu/tamil_synthetic_ocr")

In [7]:
print(dataset)

DatasetDict({
    data: Dataset({
        features: ['image', 'text'],
        num_rows: 7000
    })
})


In [8]:
train_dataset = dataset["data"]

In [9]:
train_dataset

Dataset({
    features: ['image', 'text'],
    num_rows: 7000
})

In [10]:
results = []
for i in tqdm(range(len(dataset['data']))):
    item = dataset['data'][i]
    image = item['image']  # PIL Image object
    reference_text = item['text']

    # Convert PIL Image to bytes (PNG format for consistency)
    image_bytes = BytesIO()
    image.save(image_bytes, format="PNG")
    image_content = image_bytes.getvalue()

    generated_text = run_quickstart(image_content)
    results.append({'reference': reference_text, 'generated': generated_text})

  7%|▋         | 511/7000 [06:02<1:35:45,  1.13it/s]

Error during OCR: 503 Getting metadata from plugin failed with error: HTTPSConnectionPool(host='oauth2.googleapis.com', port=443): Max retries exceeded with url: /token (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1000)')))


100%|██████████| 7000/7000 [1:24:35<00:00,  1.38it/s] 


In [11]:
results

[{'reference': 'இங்கச ர்ந்தமழப கப்படம எடத்தப்பதமக்கயம் இல்லய',
  'generated': 'இங்கச ர்ந்தமழப கப்படம் எடத்தப்பதமக்கயம் இல்லய'},
 {'reference': 'ஆண்ட உனக்கபதத இல்ல ஆனல்நீ இம்மறகற்றஞ்சட்டப்பட்டவரன்பக்கம்',
  'generated': 'ஆண்ட உனக்கபதத இல்ல ஆனல்நீ இம்மறகற்றஞ்சட்டப்பட்டவரன்பக்கம்'},
 {'reference': 'பர்களம்தயர் எல்லரம்தய ர் கபத்தல்கந்தளக்கம்',
  'generated': 'பர்களம்தயர் எல்லரம்தய ர் கபத்தல்கந்தளக்கம்'},
 {'reference': 'அவர் எங்களடன்பணயற்றத்தடங்கம்பதஅவர்கள்சய்வர்கள்',
  'generated': 'அவர் எங்களடன்பணயற்றத்தடங்கம்பதஅவர்கள்சய்வர்கள்'},
 {'reference': 'நான் நாளை ஒரு சிகிச்சை வேண்டும் நீங்கள் அதை செய்ய முடியும்',
  'generated': 'நான் நாளை ஒரு சிகிச்சை வேண்டும் நீங்கள் அதை செய்ய முடியும்'},
 {'reference': 'நன்உங்கள்தய்க்கஅளத்த வக்கறத யநறவற்றமயற்ச க்கறன்',
  'generated': 'நன்உங்கள்தய்க்க அளத்த வக்கறத யநறவற்றமயற்ச க்கறன்'},
 {'reference': 'மன்னள் மறயஎஸ்ச னட்உற ப்பனர் மற்றம்பனப்பர்பரள',
  'generated': 'மன்னள் மறயஎஸ்ச னட்உற ப்பனர் மற்றம்பனப்பர்பரள'},
 {'reference': 'தலஆறவக்கறதக்ககசரயனப்ப ய் பர்ப்பமஞ

In [12]:
df = pd.DataFrame(results)

In [13]:
df.head(20)

,reference,generated
0,இங்கச ர்ந்தமழப கப்படம எடத்தப்பதமக்கயம் இல்லய,இங்கச ர்ந்தமழப கப்படம் எடத்தப்பதமக்கயம் இல்லய
1,ஆண்ட உனக்கபதத இல்ல ஆனல்நீ இம்மறகற்றஞ்சட்டப்பட்...,ஆண்ட உனக்கபதத இல்ல ஆனல்நீ இம்மறகற்றஞ்சட்டப்பட்...
2,பர்களம்தயர் எல்லரம்தய ர் கபத்தல்கந்தளக்கம்,பர்களம்தயர் எல்லரம்தய ர் கபத்தல்கந்தளக்கம்
3,அவர் எங்களடன்பணயற்றத்தடங்கம்பதஅவர்கள்சய்வர்கள்,அவர் எங்களடன்பணயற்றத்தடங்கம்பதஅவர்கள்சய்வர்கள்
4,நான் நாளை ஒரு சிகிச்சை வேண்டும் நீங்கள் அதை செ...,நான் நாளை ஒரு சிகிச்சை வேண்டும் நீங்கள் அதை செ...
5,நன்உங்கள்தய்க்கஅளத்த வக்கறத யநறவற்றமயற்ச க்கறன்,நன்உங்கள்தய்க்க அளத்த வக்கறத யநறவற்றமயற்ச க்கறன்
6,மன்னள் மறயஎஸ்ச னட்உற ப்பனர் மற்றம்பனப்பர்பரள,மன்னள் மறயஎஸ்ச னட்உற ப்பனர் மற்றம்பனப்பர்பரள
7,தலஆறவக்கறதக்ககசரயனப்ப ய் பர்ப்பமஞபகம்இரக்க,தலஆறவக்கறதக்ககசரயனப்பய் பர்ப்பமஞபகம்இரக்க
8,கறந்தபட்சம்நன்மடந்ததஎன்றசந்த ஷமகஇர க்கம் ஒரவரன...,கறந்தபட்சம்நன்மடந்ததஎன்றசந்த ஷமகஇர க்கம் ஒரவரன...
9,நீங்கள்தன்அந்தமர ந்த எழதக்கண்டரப்பீர்கள்எனநனக்...,நீங்கள்தன்அந்தமரந்த எழுதக்கண்டரப்பீர்கள்எனநனக்...


In [14]:
df.to_csv("Google_Cloud_Vision_OCR_Results.csv", index = False)

In [15]:
from jiwer import wer, cer

In [16]:
agg_error_wer = 0
agg_error_cer = 0
for i in range(len(df)):
    record = df.iloc[i]
    try:
        error_wer = wer(record['reference'], record['generated'])
        error_cer = cer(record['reference'], record['generated'])
    except:
        print(record)
        print(i)
    agg_error_wer += error_wer
    agg_error_cer += error_cer

In [17]:
print(f"Word Error Rate (WER): {agg_error_wer/ len(df)}")

Word Error Rate (WER): 0.120576331208474


In [18]:
print(f"Character Error Rate (CER): {agg_error_cer/ len(df)}")

Character Error Rate (CER): 0.008009779731538295
